In [1]:
from google.colab import drive
drive.mount('/Til')

Mounted at /Til


In [2]:
!pip install transformers sentencepiece sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.2 MB/s eta 0:00:00


In [3]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

In [ ]:
token_path = 

In [ ]:
tokenizer.lang_code_to_id['qm_XX'] = old_len-1
tokenizer.id_to_lang_code[old_len-1] = 'qm_XX'
tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
if 'qm_XX' not in tokenizer.additional_special_tokens:
    tokenizer.additional_special_tokens.append('qm_XX')

In [10]:
model_ru_qm_path = '/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-ru-qm-v1'
model_qm_ru_path = '/Til/MyDrive/Colab Notebooks/Til/models/qm/mbart-large-52-qm-ru-v1'

In [24]:
tokenizer_ru_qm = MBart50Tokenizer.from_pretrained(model_ru_qm_path)
tokenizer_qm_ru = MBart50Tokenizer.from_pretrained(model_qm_ru_path)


In [ ]:
model_ru_qm = MBartForConditionalGeneration.from_pretrained(model_ru_qm_path)
model_qm_ru = MBartForConditionalGeneration.from_pretrained(model_qm_ru_path)

In [25]:
def fix_tokenizer(tokenizer):
    old_len = len(tokenizer) - int('qm_XX' in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id['qm_XX'] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = 'qm_XX'
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if 'qm_XX' not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append('qm_XX')
    tokenizer.added_tokens_encoder = {}

In [26]:
fix_tokenizer(tokenizer_ru_qm)
fix_tokenizer(tokenizer_qm_ru)

In [28]:
def translate(text, model, tokenizer, src='ru_RU', trg='qm_XX', max_length='auto', num_beams=3, repetition_penalty=5.0, train_mode=False, n_out=None, **kwargs):
    tokenizer.src_lang = src
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
    if max_length == 'auto':
        max_length = int(32 + 1.5 * encoded.input_ids.shape[1])
    if train_mode:
        model.train()
    else:
        model.eval()
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[trg], 
        max_length=max_length, 
        num_beams=num_beams,
        repetition_penalty=repetition_penalty,
        # early_stopping=True,
        num_return_sequences=n_out or 1,
        **kwargs
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    if isinstance(text, str) and n_out is None:
        return out[0]
    return out

In [31]:
translate("Не этесе?", model_qm_ru, tokenizer_qm_ru, src = 'qm_XX', trg='ru_RU')

'Что ты делаешь?'

In [32]:
translate("Что делаешь?", model_ru_qm, tokenizer_ru_qm, src = 'ru_RU', trg='qm_XX')

'Не этесе?'